In [ ]:
# %load ../snippets/basic_settings.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


sns.set_context("notebook", font_scale=1.1)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
plt.rcParams["figure.figsize"] = (16, 12)
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.autolayout'] = False
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['text.usetex'] = False  # True activates latex output in fonts!
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

In [ ]:
import scipy
from  statsmodels.stats.multitest import multipletests
import subprocess
import os

# Loading the data

In [ ]:
root = Path("/nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq")
dataDir = root/"scratch/08_21/counts/skroon/dnaid2030"
controls_file = root/"data/metadata/controls.txt"
outDir = root/"scratch/08_21/results/skroon"
cnt_file = outDir/'21-10-2021-annotated_gene_counts_after_qc.csv'
filt_cnt_file = outDir/'21-10-2021-annotated_gene_filtered_counts_after_qc.csv'

## Set up data for DESeq2 analysis
1. Sample Data
2. Expression Data (count table)
3. Design formula

In [ ]:
# Takes already pre-filtered dataset, and sets up input for DESeq2
# Assume looked through # of samples etc and specified correct design

def generate_DE_dataset(exp_df, sample_id='sampleID', feat_id = 'ShortName', cnt_col='barcode_cnt',  
                        other_cols = ('mouse', 'day', 'tissue', 'dnaid', 'experiment')):

    sample_data = (exp_df[[sample_id] +[c for c in other_cols]]
                   .set_index(sample_id).drop_duplicates())
    
    expr_data = (exp_df[[sample_id, feat_id, cnt_col]].drop_duplicates()
                 .pivot(index=feat_id, columns=sample_id)
                 .fillna(0))
    
    expr_data.columns = [c[1] for c in expr_data.columns]
    expr_data = expr_data[list(sample_data.index)].reset_index()

    return sample_data, expr_data

In [ ]:
sdf, edf = generate_DE_dataset(cnts, other_cols=['mouse', 'day', 'tissue'])

# Run DESeq Script

In [ ]:
def run_command(args):
    """Run command, transfer stdout/stderr"""
    result = subprocess.run(args)
    try:
        result.check_returncode()
    except subprocess.CalledProcessError as e:
        raise e
        

def get_fitness_results(fitness_dir, experiment, sdf, edf, design, r_path, feat_id):
    sdf_path = Path(fitness_dir) / f"{experiment}_sdf.csv"
    edf_path = Path(fitness_dir) / f"{experiment}_edf.csv"
    sdf.to_csv(sdf_path)
    edf.set_index(feat_id).to_csv(edf_path)
    cmd = f'Rscript {r_path} {sdf_path} {edf_path}'
    print(cmd)
    r = run_command(cmd.split())
    os.remove(sdf_path)
    os.remove(edf_path)

In [ ]:
get_fitness_results(outDir, 'skroon_test', sdf, edf, 'tissue', 
                    root/"code/notebooks/10_21_skroon/DEseq.R", 'ShortName')

In [ ]:
feces_results = (pd.read_csv(root/'scratch/08_21/results/skroon/21-10-2021-skroon_feces_results.csv')
                 .assign(tissue='feces')
                 .rename({'Unnamed: 0': 'barcode'}, axis=1))
cc_results = (pd.read_csv(root/'scratch/08_21/results/skroon/21-10-2021-skroon_cc_results.csv')
              .assign(tissue='cc')
              .rename({'Unnamed: 0': 'barcode'}, axis=1))


# Compare to WT 

In [ ]:
def sigma(lfcSE):
    return np.sqrt(lfcSE.pow(2).sum()) / len(lfcSE)


def calculate_2dist_zscore(u1, s1, u2, s2):
    return (u1 - u2) / np.sqrt((s1 ** 2) + (s2 ** 2))


def to_list(x):
    bc_list = list(x)
    if len(bc_list) == 1:
        return bc_list[0]
    return ", ".join(list(set(x)))


def calculate_comparisons2(fitness, df, control_file, treat_var='day'):
    """

    fitness: DESeq2 output, log2FoldChange value for each barcode comparing each time point with inoculum
    df: df for 1 experiment and 1 dnaid
    controls: control meta df?
    """
    days = sorted(list(fitness[treat_var].unique()))
    # days.remove('d0')
    controls = pd.read_table(control_file, names=['barcode', 'phenotype', 'conc'])

    controls['CntrlName']= controls['phenotype'] +'-'+ controls['conc'].astype(str)
    controls_bc = controls[controls.phenotype == 'wt'].CntrlName.values
    #controls_bc = controls[controls.phenotype == 'wt'].barcode.values
    cntrl_df = fitness[fitness.barcode.isin(controls_bc)]

    gene_df = fitness[~fitness.barcode.isin(controls_bc)].rename({'barcode':'ShortName'}, axis=1)
    gene_mean = gene_df.groupby(['ShortName', treat_var]).agg(
            {'log2FoldChange': ['mean', 'median'], 'lfcSE': [sigma]}).reset_index()
    gene_mean.columns = ['gene', treat_var, 'gene_FC', 'gene_FC_median', 'sigma']
    cntrl_mean = cntrl_df.groupby([treat_var]).agg({'log2FoldChange': ['mean', 'median'], 'lfcSE': [sigma]})
    cntrl_mean.columns = ['cntrl_FC', 'cntrl_FC_median', 'cntrl_sigma']
    cntrl_mean = cntrl_mean.reset_index()
    gene_mean = gene_mean.merge(cntrl_mean, how='left', on=treat_var)
    
    gene_mean['zscore'] = gene_mean.apply(
            lambda x: calculate_2dist_zscore(x['gene_FC'], x['sigma'], x['cntrl_FC'], x['cntrl_sigma']), axis=1)

    gene_mean['ci'] = gene_mean.apply(lambda x: 2 ** x['gene_FC'] / 2 ** x['cntrl_FC'], axis=1)
    gene_mean = gene_mean[['gene', treat_var, 'gene_FC',  'sigma', 'zscore', 'ci']]
    results = gene_mean.copy()
    results['pval'] = results.zscore.apply(lambda x: scipy.stats.norm.sf(abs(x)) * 2)
    results['padj'] = results.groupby(treat_var).pval.transform(lambda x: multipletests(x, alpha=0.05, method='fdr_bh')[1])
    return results


In [ ]:
results = calculate_comparisons2(fitness_df, cnts, controls_file, treat_var='tissue')

In [ ]:
zresults_feces = results[results.tissue == 'feces']
zresults_cc = results[results.tissue == 'cc']

In [ ]:
zresults_cc.to_csv(root/'scratch/08_21/results/skroon/21-10-2021-skroon_cc_zscore_results.csv')
zresults_feces.to_csv(root/'scratch/08_21/results/skroon/21-10-2021-skroon_feces_zscore_results.csv')